In [1]:
import numpy as np
import pandas as pd
from pysr import PySRRegressor
from IPython.display import HTML
from matplotlib import pyplot as plt

In [50]:
df_clean = pd.read_csv('clean_table_MBH.csv')

In [3]:
nanpercent=0.16
cols_to_delete = df_clean.columns[df_clean.isnull().sum()/len(df_clean) > nanpercent]
df_lessnan=df_clean.drop(cols_to_delete, axis = 1)
df_purge=df_lessnan.dropna(axis='index',how='any')
print(df_purge.columns)
print(len(df_purge.columns))
#HTML(df_purge.to_html())

Index(['T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge', 'M_BH',
       'M_BH_std', 'M*_sph', 'M*_gal', 'log_B/T', 'log_sigma0',
       'log_R_e_sph_maj', 'log_R_e_sph_eq_kpc', 'log_n_sph_maj',
       'log_n_sph_eq', 'log(I_e,sph,maj/M_Sun/pc^2)',
       'log(I_e,sph,eq/M_Sun/pc^2)', 'Concentration_Index',
       'avg_Rho_1kpc_Exact_All', 'r1_density_approx', 'log10(R10_kpc)',
       'logRho_R10_approx', 'log_rho10_Exact', 'log10(R90_kpc)',
       'logRho_R90_approx', 'log_rho_90_Exact_all', 'Rho_re_spatial',
       'SR_pc_All', 'Rho_SR_pc_All', 'CR_def1_approx_new',
       'Rho_cr_def1_approx_new', 'CR_def2_approx_new',
       'Rho_CR_def2_approx_new', 'Sr(pc)_2_using_Falserm_drho',
       'Log_Approx_Avg_density_10pc', 'log_Rho_e_Exact_new',
       'logRho_e_approx_New', 'logRho_soi_approx_new', 'log_Rho_soi_exact_new',
       'Avg_Rho_Re_Exact_all', 'Avg_Rho_soi_exact_all', 'Avg_Rho_re_Exact_all',
       'Rho_re_Exact_all', 'r_soi_2BH_approx', 'Rho_r_soi_2BH_approx',
       'Lo

In [5]:
## useful functions
def x2name(feature_list,df_handson):
    for ind in feature_list:
        i=int(ind[1:])
        print('x',i,':',df_handson.columns[i])

def rmse(y,y_pred,w):
    return np.sqrt(np.average((y-y_pred)**2,weights=w))

def df2name(df):
    for i in range(len(df.columns)):
        print('x',i,':',df.columns[i])

In [70]:
def test_relation(df,operator='simp',ncyclesperiteration=550,niterations=40,denoise=False,adaptive_parsimony_scaling=20,verbosity=0):
    y = df_purge['M_BH'].to_numpy()
    w = 1/df_purge['M_BH_std'].to_numpy()**2

    #df_handson = df
    X = df.to_numpy()
    #X = df_handson

    if operator=='adv':
        model = PySRRegressor(
            binary_operators=["+", "-", "*", "/","pow"],
            unary_operators=["exp","log10", "tan"],
            warm_start=False,
            denoise=denoise,
            niterations=niterations,
            ncyclesperiteration=ncyclesperiteration,
            adaptive_parsimony_scaling=adaptive_parsimony_scaling,
            verbosity=verbosity,
            precision=64,
        )
    if operator=='simp':
        model = PySRRegressor(
            binary_operators=["+", "-", "*", "/", "pow"],
            warm_start=False,
            denoise=denoise,
            niterations=niterations,
            ncyclesperiteration=ncyclesperiteration,
            adaptive_parsimony_scaling=adaptive_parsimony_scaling,
            verbosity=verbosity,
            precision=64,
        )
        
    if operator=='basic':
        model = PySRRegressor(
            binary_operators=["+", "-", "*", "/"],
            warm_start=False,
            denoise=denoise,
            niterations=niterations,
            ncyclesperiteration=ncyclesperiteration,
            adaptive_parsimony_scaling=adaptive_parsimony_scaling,
            verbosity=verbosity,
            precision=64,
        )



    model.fit(X=X, y=y, weights=w)

    print('parameters:')
    df2name(df)
    
    print('Eq. selected rmse:',rmse(y,model.predict(X),w))
    display(model.sympy())
    
    for i in range(len(model.equations_)):
        print('Eq.',i,'rmse:',rmse(y,model.predict(X,index=i),w))
        display(model.sympy(index=i))

In [9]:
df_purge.columns

Index(['T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge', 'M_BH',
       'M_BH_std', 'M*_sph', 'M*_gal', 'log_B/T', 'log_sigma0',
       'log_R_e_sph_maj', 'log_R_e_sph_eq_kpc', 'log_n_sph_maj',
       'log_n_sph_eq', 'log(I_e,sph,maj/M_Sun/pc^2)',
       'log(I_e,sph,eq/M_Sun/pc^2)', 'Concentration_Index',
       'avg_Rho_1kpc_Exact_All', 'r1_density_approx', 'log10(R10_kpc)',
       'logRho_R10_approx', 'log_rho10_Exact', 'log10(R90_kpc)',
       'logRho_R90_approx', 'log_rho_90_Exact_all', 'Rho_re_spatial',
       'SR_pc_All', 'Rho_SR_pc_All', 'CR_def1_approx_new',
       'Rho_cr_def1_approx_new', 'CR_def2_approx_new',
       'Rho_CR_def2_approx_new', 'Sr(pc)_2_using_Falserm_drho',
       'Log_Approx_Avg_density_10pc', 'log_Rho_e_Exact_new',
       'logRho_e_approx_New', 'logRho_soi_approx_new', 'log_Rho_soi_exact_new',
       'Avg_Rho_Re_Exact_all', 'Avg_Rho_soi_exact_all', 'Avg_Rho_re_Exact_all',
       'Rho_re_Exact_all', 'r_soi_2BH_approx', 'Rho_r_soi_2BH_approx',
       'Lo

stellar mass: $M*_{sph}$,$M*_{gal}$

In [14]:
test_relation(df_purge[['M*_sph','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']])

/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Cycles per second: 2.780e+04
Head worker occupation: 67.4%. This is high, and will prevent efficient resource usage. Increase `ncyclesperiteration` to reduce load on head worker.
Progress: 109 / 600 total iterations (18.167%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.643e-01  -1.000e-10  8.181178431654514
3           1.946e-01  7.455e-01  (x0 - 2.3184574449396607)
5           1.869e-01  2.004e-02  ((x0 - 1.0802338576670358) + -1.3256614205761683)
7           1.689e-01  5.079e-02  ((x4 + (x0 * 0.9068924255978891)) + -1.5694401906401392)
9           1.606e-01  2.523e-02  ((x6 * -0.1760846068747684) + ((x0 - 0.30040174212974663) + -2.063104829043233))
11          1.384e-01  7.433e-02  (((x0 - (x4 * -0.5181275544704662)) + x4) - (x4 + 2.5287170120162297))
13          1.119e-01  1.065e-01  (x0 - (0.055505940430061504 - ((x4 * 0.40270825510148694) + ((x6 * -0.416820476688901) + -2.3577917715326873))))
15         

x0 + 0.4513427388345597*x4 - 0.4513427388345597*x6 - 2.3771795900239576

Eq. 0 rmse: 0.9296505481997018


8.181178431412174

...

x0 + x4*(0.512613706699028*x3 + 0.32752016478105167) - 0.43853958643458657*x6 - 2.401367393608008

Eq. 8 rmse: 0.32401688938384493


x0 + x4*(x3 + 0.3050089258949896) + x6*(-0.3050089258949896 - 0.3508326417546147/x1) - 2.400821287350342

Eq. 9 rmse: 0.3178776240269848


x0 + x4*(0.5903083100245906*x3 + 0.3050089258949896) + x6*(-0.3050089258949896 - 0.3508326417546147/x1) - 2.397950642002973

Bulge Equivalent Properties: $n_{eq,sph}$,$C(1/3)$,$R_{e,sph,eq}$

In [15]:
test_relation(df_purge[['log_R_e_sph_eq_kpc','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']])

/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Cycles per second: 7.250e+04
Head worker occupation: 24.9%. This is high, and will prevent efficient resource usage. Increase `ncyclesperiteration` to reduce load on head worker.
Progress: 225 / 600 total iterations (37.500%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.643e-01  -1.000e-10  8.181178431641339
3           3.574e-01  4.415e-01  (x0 - -7.979806199773363)
5           1.964e-01  2.994e-01  (x0 - (-7.742756977600018 - x4))
7           1.945e-01  4.923e-03  (((x0 * 0.9117671698018492) + x4) - -7.760524619519772)
9           1.895e-01  1.302e-02  (((x4 - -7.82548195702211) - (x2 * 0.16639908062892134)) + x0)
11          1.674e-01  6.188e-02  ((x4 + (x0 + 7.742756977606434)) + ((x2 - x3) * -0.399517908965665))
15          1.446e-01  3.657e-02  (((x0 - ((x2 - x3) / ((x6 + -0.12622294910246246) * -10.611300986670265))) + 7.742756977606434) + x4)

Press 'q' and then <enter> to stop execution early.

Cycles

x0 + x4 + 7.742756977600848

Eq. 0 rmse: 0.9296505481997018


8.181178431641339

...

x0 + x4 + 7.742756977606434 - (x2 - x3)/(1.3393897043513905 - 10.611300986670265*x6)

Eq. 8 rmse: 0.3776579227935076


x0 + x4 + 0.43258069677255573*(-0.7210334122339703*x2 + 0.7210334122339703*x3)/(x6 + 0.5818054262914165) + 7.65504409920661

Eq. 9 rmse: 0.36347759091842774


x0 + x4 - (-x2 + x3)*(x6 - 0.7087016791839875) + 7.6472129079794273

In [16]:
test_relation(df_purge[['log_n_sph_eq','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']])

/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Cycles per second: 4.510e+04
Head worker occupation: 8.9%
Progress: 130 / 600 total iterations (21.667%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.643e-01  -1.000e-10  8.181178431414043
3           3.999e-01  3.853e-01  (x4 + 7.944129209237331)
5           2.611e-01  2.132e-01  ((x4 + 8.441277265045823) - x2)
7           2.487e-01  2.424e-02  ((x4 - (x2 * 0.7776742066467353)) + 8.330748429121124)
9           2.254e-01  4.918e-02  ((x4 + 8.505558772342981) + (-0.7570478736273929 * (x2 + x6)))
11          2.209e-01  1.018e-02  ((x4 + 8.505558772342981) + ((-0.7570478736273929 + 0.05765287043969508) * (x2 + x6)))
13          1.980e-01  5.473e-02  ((x4 - (((x5 * x2) * (x3 - x6)) + x6)) + 8.394366234198158)
17          1.909e-01  9.116e-03  ((x4 - (((((x2 - x6) + x1) * 0.10399821834085572) * (x5 - x6)) + x6)) + 8.238916212579852)
19          1.879e-01  8.016e-03  (((((((x2 + -0.6590371465870319) * (x6 - x4)) - (

-x2*(0.8324917713612128 - x6) + x4 - x6 + 8.394366234637316

Eq. 0 rmse: 0.9296505481997017


8.18117843141749

...

x4 - x6 - (0.9713578119099814 - x6)*(0.8051064362296408*x2 - 0.09591033006752602*x4) + 8.394366234198158

Eq. 9 rmse: 0.41729334068674884


x4 - x6 - (0.9713578119099814 - x6)*(0.8051064362296408*x2 - 0.8051064362296408*x4*(0.21175917419706017 - x6)) + 8.394366234198158

Eq. 10 rmse: 0.4130645418056561


-x0*x2*(x5*(x0 - 2*x6) - x6 + 1.4367533792119422) + x4 - x6 + 8.400941036820713

velocity dispersion: $\sigma$

In [17]:
test_relation(df_purge[['log_sigma0','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']])

/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Cycles per second: 4.260e+04
Head worker occupation: 7.1%
Progress: 139 / 600 total iterations (23.167%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.643e-01  -1.000e-10  8.181178431411102
3           2.465e-01  6.273e-01  (x0 / 0.2712231392260813)
5           1.611e-01  2.127e-01  ((x0 / 0.2712231392260813) - x6)
7           1.423e-01  6.193e-02  ((x0 * 3.689099720222239) - (x6 / 1.4196753003327522))
9           1.370e-01  1.889e-02  (((2.2828586461816758 + (x0 * x0)) - x6) + 0.9750127622572914)
11          1.029e-01  1.435e-01  ((x0 / 0.2809883042690955) + ((x4 + -0.36315577132759436) * (x6 + x4)))
13          1.021e-01  3.521e-03  ((x0 / 0.2809883042690955) + ((x4 + -0.36315577132759436) * ((x6 * 1.143023342600646) + x4)))
15          9.284e-02  4.767e-02  ((x0 / 0.2712231392260813) + (((x3 - (x4 * x0)) * -0.1619521928624439) - (x6 * 0.5163955850760338)))
17          9.213e-02  3.862e-03  ((x0 / 0.271223139

3.558866987724603*x0 + 0.6066361437807702*x4 - 0.6066361437807702*x6

Eq. 0 rmse: 0.9296505481997018


8.181178431411102

...

0.1619521928624439*x0*x4 + 3.687001053278268*x0 - 0.1619521928624439*x3 - 0.5163955850760338*x6

Eq. 8 rmse: 0.3035284138135658


0.1619521928624439*x0*x4 + 3.687001053278268*x0 - 0.1619521928624439*x3 - 0.4963592082197769*x6 - 0.021473370366360859

Eq. 9 rmse: 0.302680460514895


0.17096299318790864*x0*x4 + 3.65775709400353*x0 - 0.17096299318790864*x3 - 0.17096299318790864*x6*(x0 + 0.6364977740268811) + 0.023061398742541962

In [31]:
test_relation(df_purge[['log_sigma0', 'Core', 'Pseudobulge']],adaptive_parsimony_scaling=50)

/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Cycles per second: 4.430e+04
Head worker occupation: 8.6%
Progress: 135 / 600 total iterations (22.500%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.643e-01  -1.000e-10  8.18117843141525
3           4.006e-01  3.844e-01  (x1 + 7.9699142303007)
5           1.685e-01  4.330e-01  ((x0 * x0) - -3.0409347240893494)
7           1.370e-01  1.035e-01  ((x0 * x0) - (x2 + -3.252198925196203))
9           1.085e-01  1.167e-01  ((x0 * (x0 - (0.27431474747550777 * x2))) - -3.167611367125819)
15          1.073e-01  1.747e-03  (((x0 / 0.39268792606912856) * 1.0191218622616016) + (x0 + (0.39268792606912856 * ((x1 ^ x0) - x2))))
17          1.031e-01  2.044e-02  ((((x0 * x0) + (x2 * -0.5821051775379205)) - (x0 * ((x1 * -0.06287119293836549) * 1.5010444790456636))) - -3.167611367125819)
19          9.887e-02  2.073e-02  ((((x0 * x0) + (x2 * (x1 - 0.5180930076759438))) - (x0 * ((x1 * -0.04102484694715318) * x0))) - -3.090113188

x0*(x0 - 0.27431474747550777*x2) + 3.167611367125819

Eq. 0 rmse: 0.9296505481997018


8.18117843141525

...

2*x0 + x0**(0.7892999392426963**x2)*(1.2244251512504083**x1 + 0.6096726597332394)

Eq. 7 rmse: 0.3056745627307154


0.8482357326566811**x2*x0*(x0**(0.22289005172419304*x1) + 0.6091686277688106) + 2*x0

Eq. 8 rmse: 0.3038876030876562


2*x0 + x0**(0.7895384209691106**x2)*((x0 - x1)**(0.51296581455262264*x1) + 0.6093711018229226)

Ratio of light: $B/T$,$D/T$

In [18]:
test_relation(df_purge[['log_B/T','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']])

/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Cycles per second: 4.900e+04
Head worker occupation: 7.4%
Progress: 158 / 600 total iterations (26.333%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.643e-01  -1.000e-10  8.181178431413086
3           3.999e-01  3.853e-01  (x4 + 7.94412920923932)
5           1.973e-01  3.532e-01  ((x4 + x0) + 8.37638528788242)
7           1.811e-01  4.302e-02  ((x4 + 8.538398716270097) + (x0 * 1.374808907288754))
11          1.803e-01  1.083e-03  (((x0 - ((x6 - -0.4265959363555742) * 0.31989230471046565)) + 8.580431837503703) + x4)
13          1.796e-01  1.808e-03  ((x4 + 8.506654538189625) + ((x3 * 1.1389505658019166) - (x3 + (x0 * -1.5257095099277))))
15          1.732e-01  1.833e-02  (((x4 + 8.538398716270097) + ((x6 / 0.6403017171166149) * -0.17561946650369467)) + (x0 - (x2 * 0.17831139813207694)))
17          1.662e-01  2.049e-02  (((x0 - (((x6 - (1.049451639251683 * x0)) * (x0 + 1.2447771054033887)) * 0.49868185417201455

x0 + x4 + 8.37638528788242

Eq. 0 rmse: 0.9296505481997018


8.181178431413086

...

1.2281729749477047*x0 + x4 - 0.22817297494770467*x6 + 8.523219224445711

Eq. 6 rmse: 0.3969252314301504


x0*(0.39116603969655633 - x6) + x0 + x4 - x6 + 8.5626626784275

Eq. 7 rmse: 0.3845258275634705


x0*(-6.364633613299979*x3 - x6 + 6.802421613543683) + x0 + x4 - x6 + 8.645567605765882

Densities

In [20]:
df_purge.columns

Index(['T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge', 'M_BH',
       'M_BH_std', 'M*_sph', 'M*_gal', 'log_B/T', 'log_sigma0',
       'log_R_e_sph_maj', 'log_R_e_sph_eq_kpc', 'log_n_sph_maj',
       'log_n_sph_eq', 'log(I_e,sph,maj/M_Sun/pc^2)',
       'log(I_e,sph,eq/M_Sun/pc^2)', 'Concentration_Index',
       'avg_Rho_1kpc_Exact_All', 'r1_density_approx', 'log10(R10_kpc)',
       'logRho_R10_approx', 'log_rho10_Exact', 'log10(R90_kpc)',
       'logRho_R90_approx', 'log_rho_90_Exact_all', 'Rho_re_spatial',
       'SR_pc_All', 'Rho_SR_pc_All', 'CR_def1_approx_new',
       'Rho_cr_def1_approx_new', 'CR_def2_approx_new',
       'Rho_CR_def2_approx_new', 'Sr(pc)_2_using_Falserm_drho',
       'Log_Approx_Avg_density_10pc', 'log_Rho_e_Exact_new',
       'logRho_e_approx_New', 'logRho_soi_approx_new', 'log_Rho_soi_exact_new',
       'Avg_Rho_Re_Exact_all', 'Avg_Rho_soi_exact_all', 'Avg_Rho_re_Exact_all',
       'Rho_re_Exact_all', 'r_soi_2BH_approx', 'Rho_r_soi_2BH_approx',
       'Lo

In [21]:
test_relation(df_purge[['logRho_soi_approx_new','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']])

/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Cycles per second: 5.160e+04
Head worker occupation: 60.1%. This is high, and will prevent efficient resource usage. Increase `ncyclesperiteration` to reduce load on head worker.
Progress: 187 / 600 total iterations (31.167%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.643e-01  -1.000e-10  8.181178431301062
3           3.999e-01  3.853e-01  (x4 + 7.944129209135556)
5           2.889e-01  1.627e-01  (7.758365983265747 - (x4 * -1.783648309316931))
7           2.634e-01  4.622e-02  (8.181178431411562 + (x4 - (-0.21256840944483552 + x2)))
9           2.426e-01  4.111e-02  ((x4 + 7.885305031739153) + ((x5 - x1) / 8.973798199391117))
11          2.388e-01  7.929e-03  ((x4 + 7.944129208906086) + ((x4 - x1) / (x0 + 7.944129208906086)))
13          2.287e-01  2.155e-02  (((x6 * -0.269060969796932) + 8.036078589458995) + (x4 - ((x1 * 0.5322656254986011) * 0.14509984195256675)))
15          1.988e-01  7.009e-02  (((((x0

x4 + x6*(x2 - 1.1598775212786263) + 5.41147838328439*(1.443193286119009*x0 - 0.33435867154210816*x2 + 0.23167975818488953)/x0

Eq. 0 rmse: 0.9296505481997018


8.181178431301062

...

-0.4361747248793917*x0 - 0.64554825365805757*x2 + x4 + 9.437355490458047

Eq. 9 rmse: 0.39809646459506504


-0.4361747259132775*x0 - 0.64554825478674843*x2 + x4 + 9.4373554939610405

Eq. 10 rmse: 0.3642217411034978


x4 + x6*(x2 - 1.1598775212786263) + 5.41147838328439*(1.443193286119009*x0 - 0.33435867154210816*x2 + 0.23167975818488953)/x0

In [22]:
test_relation(df_purge[['Log_Avg_Rho_5kpc_approx','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']])

/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Cycles per second: 3.030e+04
Head worker occupation: 91.4%. This is high, and will prevent efficient resource usage. Increase `ncyclesperiteration` to reduce load on head worker.
Progress: 135 / 600 total iterations (22.500%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.643e-01  -1.000e-10  8.18117843141749
3           3.261e-01  4.873e-01  (x0 - -9.553550099159985)
5           1.491e-01  3.912e-01  ((x0 + 9.319035394610406) + x4)
7           1.491e-01  9.087e-05  ((x0 + (x4 * 0.9893080534019084)) + 9.319035394610406)
9           1.276e-01  7.790e-02  (x0 + ((9.319035394610406 - (x6 * 0.2873079448687995)) + x4))
11          1.226e-01  1.998e-02  ((((x0 + 0.05500909788246669) + x4) - (0.36215667350336705 * x6)) + 9.319035394610406)
13          1.222e-01  1.454e-03  (((((x0 - -0.020939499809726363) + 0.05500909788246669) + x4) - (x6 * 0.36215667350336705)) + 9.319035394610406)
15          1.219e-01  1.202e-03  (

x0 + x4 + 9.316500877101788

Eq. 0 rmse: 0.9296505481997017


8.18117843141749

...

x0 + x4 - 2.21303817922197*x6/(6.830654191548931 - 2.341291582523514/x1) + 9.40181937354928

Eq. 8 rmse: 0.3331275699663772


x0 - 0.2373845286702575*x2 + 0.7626154713297425*x4 - 0.39040737022249694*x6 + 9.573267052954664

Eq. 9 rmse: 0.32810670136640835


x0 + 0.3004874722635362*x2*x6 - 0.3004874722635362*x2 + 0.6995125277364638*x4 - 0.6208702942892388*x6 + 9.6188422777960151

LTG(sprial) galaxies

In [53]:
nanpercent=0.7
cols_to_delete = df_clean.columns[df_clean.isnull().sum()/len(df_clean) > nanpercent]
df_lessnan=df_clean.drop(cols_to_delete, axis = 1)
df_purge=df_lessnan.dropna(axis='index',how='any')
print(df_purge.columns)
print(len(df_purge.columns),len(df_purge))
#HTML(df_purge.to_html())

Index(['T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge', 'M_BH',
       'M_BH_std', 'M*_sph', 'M*_disk', 'M*_gal', 'log_B/T', 'log_D/T',
       'm_#_of_arms', 'phi', 'log_sigma0', 'log_v_max', 'log_R_e_sph_maj',
       'log_R_e_sph_eq_kpc', 'log_n_sph_maj', 'log_n_sph_eq',
       'log(I_e,sph,maj/M_Sun/pc^2)', 'log(I_e,sph,eq/M_Sun/pc^2)',
       'Concentration_Index', 'avg_Rho_1kpc_Exact_All', 'r1_density_approx',
       'log10(R10_kpc)', 'logRho_R10_approx', 'log_rho10_Exact',
       'log10(R90_kpc)', 'logRho_R90_approx', 'log_rho_90_Exact_all',
       'Rho_re_spatial', 'SR_pc_All', 'Rho_SR_pc_All', 'CR_def1_approx_new',
       'Rho_cr_def1_approx_new', 'CR_def2_approx_new',
       'Rho_CR_def2_approx_new', 'Sr(pc)_2_using_Falserm_drho',
       'Log_Approx_Avg_density_10pc', 'log_Rho_e_Exact_new',
       'logRho_e_approx_New', 'logRho_soi_approx_new', 'log_Rho_soi_exact_new',
       'Avg_Rho_Re_Exact_all', 'Avg_Rho_soi_exact_all', 'Avg_Rho_re_Exact_all',
       'Rho_re_Exact_all

$|\phi|$

In [60]:
df_purge=df_clean[['M_BH','M_BH_std','phi','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']].dropna(axis='index',how='any')
test_relation(df_purge[['phi','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']])

/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Cycles per second: 7.160e+04
Head worker occupation: 11.4%
Progress: 226 / 600 total iterations (37.667%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.268e-01  -1.000e-10  7.607431600530748
5           7.529e-02  2.757e-01  ((x0 * -0.127491375324191) + 9.202346936864185)
7           6.448e-02  7.751e-02  (((x0 + x6) * -0.127491375324191) + 9.202346936864185)
9           5.868e-02  4.714e-02  (((x0 - (x5 - x6)) * -0.127491375324191) + 9.202346936864185)
11          5.489e-02  3.337e-02  (((x0 - (x5 - (x6 / 0.6012955938423843))) * -0.127491375324191) + 9.202346936864185)
13          5.482e-02  6.122e-04  (((x0 - (x5 - (x6 / (-0.5962287871921054 - -1.1828085140343625)))) * -0.127491375324191) + 9.202346936864185)
15          5.478e-02  4.018e-04  (((x0 - (x5 - (x6 / (-0.6083952766136351 / (1.3738602283288046 / -1.1878763197129383))))) * -0.127491375324191) + 9.202346936864185)
17          5.419e-02  5.401e-03  ((

-0.12793274344286767*x0 - 0.12793274344286767*x2 - 0.12793274344286767*x2/(9.262712759179463 - x0)**x5 + 0.12793274344286767*x5 - 0.12793274344286767*x6 + 9.262712759179463

Eq. 0 rmse: 0.47628649440503695


7.607431600530748

...

-0.12264633275495064*x0 - 0.12264633275495064*x2 + 0.12264633275495064*x5 - 0.24529266550990128*x6 + 9.238248035491454

Eq. 6 rmse: 0.21908454291726245


-0.127491375324191*x0 + 0.127491375324191*(x1*(x5**x2 - 0.2435383990886446) - 1.8065159806333415)**x6 + 9.202346936864185

Eq. 7 rmse: 0.19871965124359825


-0.12793274344286767*x0 - 0.12793274344286767*x2 - 0.12793274344286767*x2/(9.262712759179463 - x0)**x5 + 0.12793274344286767*x5 - 0.12793274344286767*x6 + 9.262712759179463

$M_{DM}$ 

In [71]:
df_purge=df_clean[['M_BH','M_BH_std','M_DM','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']].dropna(axis='index',how='any')
test_relation(df_purge[['M_DM','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']],operator="basic",adaptive_parsimony_scaling=100)

/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


parameters:
x 0 : M_DM
x 1 : T-type
x 2 : Bar
x 3 : Disk
x 4 : Core
x 5 : AGN
x 6 : Pseudobulge
Eq. selected rmse: 0.2759573605260665


x0 + x4 + 8.037241350387553 - (150.93384048248672 + 0.889196734801987/(-x1 + x3))/x0

Eq. 0 rmse: 0.47392912803269754


7.610002611166966

...

x0 + x4 + 8.037241350387553 - (150.93384048248672 + 0.889196734801987/(-x1 + x3))/x0

Eq. 8 rmse: 0.2741809830769338


x0 + 6.107937784613776 - (x5 + x6 + 126.7967504330045 + 0.8912816245788976/(-x1 + x3))/x0

Eq. 9 rmse: 0.25796471776733904


2*x0 - x5 - 15.81040484017694 + 2.353468466285202*x5/x1 - 16.16263558582034/(x0*x1)

$v_{max}$

In [72]:
df_purge=df_clean[['M_BH','M_BH_std','log_v_max','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']].dropna(axis='index',how='any')
test_relation(df_purge[['log_v_max','T-type', 'Bar', 'Disk', 'Core', 'AGN', 'Pseudobulge']],adaptive_parsimony_scaling=50,operator="basic")
#test_relation(df_purge[['log_v_max']])

/home/zj448/miniconda3/lib/python3.9/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


parameters:
x 0 : log_v_max
x 1 : T-type
x 2 : Bar
x 3 : Disk
x 4 : Core
x 5 : AGN
x 6 : Pseudobulge
Eq. selected rmse: 0.4683954195381608


-0.587509060686779*x2 + x4 + 8.12222666780487

Eq. 0 rmse: 0.6622636804780176


7.77304624859739

...

0.04497527763421013*x1 - x2 + x4 - x6*(1.1826405925271102 - x2) + 8.4742300567150682

Eq. 8 rmse: 0.4069601415797293


x4 + x6**2*(x2 - 1.1355356103731322) + 8.280881285440675 - 1.5675281290696166*x2/x0

Eq. 9 rmse: 0.40407980759828416


-0.6102852689901979*x2 + x4 + 8.206051011362751 + x6/(x0*(x2*(-x5 - 1.148543150104956) - 0.4073493792141279))